# SPI Momentum

Add a description of the project here, just a brief overview. Describe the context, datasets used, and key objectives

### Table of Content
List all other chapters here to facilitate navigation

### Setup and Dependencies
Import Libraries: Create a separate cell for importing all the necessary libraries (e.g., pandas, numpy, matplotlib, etc.).
Import functions: Import all the necessary funcitons for data loading, backtesting, etc.
Set Environment Variable: 
Set cache directory (if needed):

In [25]:
import sys
import os
import pandas as pd
import numpy as np

# Navigate up two levels from the notebooks folder to reach the project root and add `src` to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)
print(project_root)

# Import from src/analysis
from src.analysis import backtest_functions

# Import from src/data_processing
#from src.data_processing import data_processing
from src.data_processing import extract_date_and_data

/Users/steve/Documents/Coding/my-project/Abnormal-Returns-in-the-Danish-Stock-Market-With-Momentum-Strategies


### Data Import and Overview

Load data here and explain what the processing does

In [29]:
# Load and clean the data, memorize this later
#cleaned_df = data_processing.load_and_clean_data()

# Display the cleaned data
#print(cleaned_df.head())

# Process the date column and extract the rest of the data
df = pd.read_csv(os.path.join(project_root, "data", "interim", "interim_data.csv"))
numericDate_daily, data = extract_date_and_data.extract_date_and_data(df)

# crate first days and last days of month
firstDayList, lastDayList = backtest_functions.getFirstAndLastDayInMonth(numericDate_daily)

# create numeric dates monhly, as EoM
numericDate_monthly = numericDate_daily.iloc[lastDayList] 


# Display the formatted date and the rest of the data
#print("Formatted Date:", formatted_date.head())
#print("Data without Date Column:", data.iloc[:, :10].head())

# crate daily returns from indecx data, ignoring invalid values
returns_daily = backtest_functions.calculate_returns(data)

# aggregate returns to monthly returns
returns_monthly = backtest_functions.aggregate_returns(returns_daily, numericDate_daily)

if isinstance(returns_monthly, np.ndarray):
    returns_monthly = pd.DataFrame(returns_monthly)
# simulate rf_monthly
rf_monthly = pd.Series(0, index=range(299))




/var/folders/8t/79dsy0zj2097l6wnskmp6qph0000gn/T/ipykernel_1888/2657336191.py:8: DtypeWarning: Columns (11,14,28,33,38,40,43,46,47,54,61,62,66,67,71,74,75,77,78,81,84,85,86,87,89,93,98,103,105,111,121,123,125,128,134,137,138,140,141,143,145,146,149,152,154,157,162,163,164,167,170,174,175,177,178,179,180,189,194,197) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(project_root, "data", "interim", "interim_data.csv"))


### Set up Basic Parameters for Analysis

Define all the variables: Lookback_period, holding_period, etc.

### Momentum Analysis
Run the analysis code here and compare results to benchmark, show performance stats. etc., show Long and Long-Short + table with most important stats

### Robustness Checks
Perform robustness checks here over different input parameters that we "arbitraily" chose: lookback period, holding period, nLong/nShort, trx cost

### Interpretation and Conclusion 

In [60]:
import os
print(os.getcwd())

/Users/steve/Documents/Coding/my-project/Abnormal-Returns-in-the-Danish-Stock-Market-With-Momentum-Strategies/notebook
